## Beautiful Soup

É uma biblioteca que ajuda a organizar a estrutura HTML de uma página web em objetos que podem ser inspecionados pelo Python. Atua na manipulação de páginas estáticas, ou seja, ignorando JavaScript. A principal diferença deste para o Selenium é que este é melhor para coleta de dados uma vez que não liga com o javascript e sim apenas com a pagina HTML ou xML.

[Doc Oficial](https://beautiful-soup-4.readthedocs.io/en/latest/)

## Desafio 2

Faça um novo levantamento usando uma página de Imóveis, exemplo: VivaReal, loft ,QuintoAndar, e calcule para pelo menos 1 dessas regiões o preço médio do m².

**Dica**: Para calcular a média vc pode utilizar a função mean() do pandas.


In [ ]:
!pip install beautifulsoup4

In [1]:
# Imports
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

%matplotlib inline

In [128]:
# Adicionando o url do site VivaReal, com a pesquisa para Ponta Negra - Natal/RN
url = 'https://www.vivareal.com.br/venda/rio-grande-do-norte/natal/bairros/ponta-negra/apartamento_residencial/'

# Usaremos o request para realizar a requisição
vivareal_response = requests.get(url)

# Conversão da resposta para HTML Usando Beautiful Soup
vivareal_soup = BeautifulSoup(vivareal_response.text, 'html.parser')

# Titulo da Página
vivareal_soup.title

<title> Apartamentos à venda em Ponta Negra, Natal - Viva Real </title>

In [129]:
## O Método prettify nos permite visualizar toda a estrutura html com todos os filhos aplicados.
print(vivareal_soup.prettify())

<!DOCTYPE html>
<html lang="pt-BR" prefix="og: https://ogp.me/ns#">
 <head>
  <meta content="5DB2670D7BB1D1D3E36C46F7C3D59380" name="msvalidate.01"/>
  <meta content="omni2c019e0" name="omniverify"/>
  <meta content="https://www.vivareal.com.br/venda/rio-grande-do-norte/natal/bairros/ponta-negra/apartamento_residencial/" property="og:url"/>
  <meta content="" property="og:type"/>
  <meta content="" property="og:image"/>
  <meta content="image/jpeg" property="og:image:type"/>
  <meta content="" property="og:image:secure_url"/>
  <meta content="Apartamentos à venda em Ponta Negra, Natal - Viva Real" property="og:title"/>
  <meta content="Mais de 710 apartamentos à venda em Ponta Negra, Rio Grande do Norte. Acesse as melhores ofertas de venda de apartamentos em Ponta Negra." property="og:description"/>
  <meta content="Mais de 710 apartamentos à venda em Ponta Negra, Rio Grande do Norte. Acesse as melhores ofertas de venda de apartamentos em Ponta Negra." name="description"/>
  <title>
  

O Método principal para encontrar elementos aqui são 

**find**
Nos permite localizar elementos por tags em conjunto com atributos

```
Assinatura: find(name, attrs, recursive, string, **kwargs)
find ## retorna um elemento procurado
find_all ## retorna uma lista com os elementos encontrados
```


**select**
Nos permite realizar a busca pelos classes e seletores do css

exemplo:
```
soup.select(".sister")
```


In [130]:
Regiao_Escolhida = vivareal_soup.find('h1', {'class': 'results-summary__title'}).text

In [131]:
print(Regiao_Escolhida)

  710   Apartamentos à venda em Ponta Negra, Natal - RN  


In [132]:
## Limitar o campo onde os imóveis estão alocados
Area_dos_Imoveis = vivareal_soup.find('section', {'class': 'results__main'})

In [133]:
print(Area_dos_Imoveis)

<section class="results__main"> <header class="results-main__summary js-summary"> <div class="results-summary__container"> <nav class="results-summary__tabs js-business-tabs"> <ul class="tab__menu js-tab-menu"> <li> <button class="tab__menu-item js-tab" data-active="" data-business="sale" data-campaign="RESULT" data-stamps="">Comprar</button> </li> <li> <button class="tab__menu-item js-tab" data-business="rent" data-campaign="RESULT" data-stamps="">Alugar</button> </li> <li> <button class="tab__menu-item js-tab" data-business="development" data-campaign="RESULT" data-stamps="">Imóveis Novos</button> </li> </ul> </nav> <div class="results-summary__wrapper js-wrapper"> <div class="js-title"> <div class="results-summary__data"> <h1 class="results-summary__title js-summary-title"> <strong class="results-summary__count js-total-records"> 710 </strong>  Apartamentos à venda em Ponta Negra, Natal - RN  </h1> </div> </div> <div class="results-summary__filters js-filters"> <ul class="summary-fi

In [134]:
## Aqui usamos o Zip para unir titulo e preço usando apenas um for.
# for title, price in zip(Area_dos_Imoveis.find_all('span', {'class':'property-card__title'}), Area_dos_Imoveis.find_all('p')):
#    print(f"{title.string}: {price.string} \n")

In [135]:
title_list = [imovel.text for imovel in Area_dos_Imoveis.find_all('span', {'class':'property-card__title'})]
preco_list = [preco.text for preco in Area_dos_Imoveis.find_all('p')]
imovel_list = pd.DataFrame({'Imóvel': title_list, 'Preço':preco_list})

In [1]:
print(imovel_list.head(10))

NameError: name 'imovel_list' is not defined

In [137]:
imovel_list['Preço'] = imovel_list['Preço'].str.replace('R$ ','', regex = False)
imovel_list['Preço'] = imovel_list['Preço'].str.replace(' /Mês','', regex = False)
imovel_list['Preço'] = imovel_list['Preço'].str.replace(' ','', regex = False)
imovel_list['Preço'] = imovel_list['Preço'].str.replace('.','', regex = False)
imovel_list['Preço'] = imovel_list['Preço'].astype(int)

In [138]:
print(imovel_list.head(5))

                                         Imóvel   Preço
0     Apartamento com 2 Quartos à Venda, 58m²    410000
1                                    Ed coral    510766
2    Apartamento com 3 Quartos à Venda, 100m²    420000
3     Apartamento com 3 Quartos à Venda, 95m²    320000
4    Apartamento com 3 Quartos à Venda, 101m²    790000


In [139]:
media_preco_imoveis = imovel_list['Preço'].mean()
print("A média de preço de%s é de R$ %.2f" % (Regiao_Escolhida, media_preco_imoveis))

A média de preço de  710   Apartamentos à venda em Ponta Negra, Natal - RN   é de R$ 448293.22
